<a href="https://colab.research.google.com/github/Kalpesh-Rajput/Netflix_Movies_and_TV_Show_Clustering/blob/main/KR_%7C_Netflix_Movies_and_TV_Show_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>